## <center> **<span style="color: red">NLP</span> steps and explnation.**

Here we will use <span style="color: #ffd21e">**Hugging face**</span> trasformers pre trained models becuase it'll take so much effort to label<br>
all of our text columns "<ins>Comments, Describtion, Title</ins>" and also we will extract categorical and numirecal<br>
features from the text columns.<br><br>

##### **Steps in detail:**
<ul>
    <li>Extract categorical features from text columns sush as:<ol> 
        <li>Contains emojies
        <li>Positive or negative
        <li>Language
        <li>Video stats range</ol>
    <li> Doing analysis for most common words in comments and titles (Word cloud)
    <li> Extracing the length of the comments, descriptions & titles
    <li> One hot encoding
    <li> Stemming the data and tokenizing it
    <li> Saving those new data into new <strong>one</strong> Df to use in the final ML model

##### **Important note:**
The data in this analysis may be a little bit biased becuase all comments is made to be at least more than <br>
$50$.

## <center> **Importing the <span style="color: red">Packeges</span>**<br>

In [1]:
import re
import os
import ast
import time
import nltk
import emoji
import pickle
import sqlite3
import svgwrite
import numpy as np
import pandas as pd
import transformers
from PIL import Image
import huggingface_hub
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from IPython.display import set_matplotlib_formats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from wordcloud import (WordCloud,
                       STOPWORDS,
                       ImageColorGenerator)

In [2]:
with open("../Data analysis/functions/z-score.pickle", "rb") as f:
    z_score = pickle.load(f)

huggingface_hub.notebook_login()
set_matplotlib_formats('pdf', 'svg')

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_15264\2025956571.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center> **Reading the <span style="color: red">data</span>**

In [3]:
TEXT_COLUMNS = ["title", "comments", "description", "channelTitle", "about"]

con = sqlite3.connect('../database.db')

df = pd.read_sql_query("""   
                        
                        SELECT *
                        FROM base_videos AS bv

                        INNER JOIN base_channels  AS bc ON
                            bc.channel_name = bv.channelTitle

                        INNER JOIN comments  AS c ON
                            c.video_id = bv.video_id""", con)

con.close()

df = df.T.drop_duplicates().T # dropping duplicated cols

In [4]:
df.sample(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,duration_in_minutes,subscribers,total_views,date,video_count,about,country,comments,like_counts,reply_counts
2826,ROtSxPudwI4,Z1 Gaming,Finally Unlocking The AQUARIUM T2 !! Planet Cr...,#planetcrafter #z1gaming\n\nA space survival ...,"['planet crafter', 'the planet crafter', 'plan...",2023-03-28 12:30:23+00:00,20916,1390,65,hd,22.78125,512000,129586042,2016-03-23 00:00:00,3080,"Hey everyone I'm Z and welcome to my channel, ...",US,In a regular progression you unlock fish farms...,0,0
1839,1RgnnTyP9rI,Grizzley Gang Gaming,Episode 25.3: The Entire City Got A Bag On My ...,TEE GRIZZLEY Playing GTA RP\nEpisode 25.3: Of ...,"['tee grizzley gta 5', 'tee grizzley', 'tee gr...",2023-03-23 01:00:10+00:00,128886,2546,203,hd,56.875,600000,187024461,2021-01-19 00:00:00,1306,TEE GRIZZLEY (GRIZZLEY WORLD OWNER):\nhttps://...,US,🤦🏽‍♂️😂😂😂😂😂 PAYING TO EAT IT IS CRAZY AF,0,0
768,Lgbl1qI6Iak,Sumo Gaming,The *RANDOM* SKIN CHALLENGE In Garena Free Fire,SUB TO BE A LEGEND - https://bit.ly/3A2cBs9\n\...,"['sumo gaming', 'garena free fire', 'free fire...",2022-03-29 05:00:08+00:00,2776845,68285,703,hd,10.03125,619000,65969205,2019-10-13 00:00:00,223,"Hello, My name is Sumo.\nThis channel make con...",IN,POTY,0,0
1590,70HDU3K1Fv8,Predator Gaming,It Lies Within | Episode 3: Level Up | Predato...,Who is friend or foe? Kya will need to figure ...,[],2021-09-28 13:00:12+00:00,392172,1830,98,hd,4.898438,42100,5997924,2017-04-18 00:00:00,203,Want to see what we've been cooking? New gamin...,US,I would like to be a predator but... what a go...,0,0
935,cvOHtIkawS8,Judo Sloth Gaming,New Super Miner Explained (Clash of Clans),"The December Update brings a New Troop, the Su...","['clash of clans', 'coc', 'clash of clans game...",2022-12-10 13:00:30+00:00,415656,19025,783,hd,8.046875,2400000,473754517,2014-02-04 00:00:00,1784,A Clash of Clans based channel focusing on hel...,GB,"Subscribe for tomorrow Sneak Peek video, as we...",296,20


##  <center> **Extract <span style="color: red">categorical</span> features**

#### *Extract emojies count per text*

In [5]:
comments_emojis_counts: list = []
title_emojis_counts: list = []
desc_emojis_counts: list = []

for comment, title, desc in zip(df["comments"], df["title"], df["description"]):
    
    comments_emojis_count: int = 0
    title_emojis_count: int = 0
    desc_emojis_count: int = 0
    
    for comment_char, title_char, desc_char in zip(comment, title, desc):
        
        if emoji.is_emoji(comment_char):
            comments_emojis_count += 1
            
        if emoji.is_emoji(title_char):
            title_emojis_count += 1
            
        if emoji.is_emoji(desc_char):
            desc_emojis_count += 1
    
    comments_emojis_counts.append(comments_emojis_count)
    title_emojis_counts.append(title_emojis_count)
    desc_emojis_counts.append(desc_emojis_count)

In [ ]:
df["comments_emojis_count"] = comments_emojis_counts
df["title_emojis_count"] = title_emojis_counts
df["desc_emojis_count"] = desc_emojis_counts

df.sample(5)

#### *Language detection*

In [ ]:
language_detector = transformers.pipeline("text-classification",
                                          model="papluca/xlm-roberta-base-language-detection",
                                          use_auth_token= True)

In [ ]:
%%timeit
print(language_detector("هذا أطول نص قد تقرأه على الإطلاق إنه طويل للغاية", max_length=128))

Looks like we will take a lot to <ins>just classify the language</ins> of each text at least: $30\times100$ second whitch equalls <span style="color: red">**50 minutes.**</span><br>
So we can't do translation for now becuase it at least may take **5 hours** to run so we will just use this language detetion and<br>
optimize it by just sellecting one text column to detect which will be **Description column** and use the detected languages to<br>
pridect video and comments language.<br><br>
But we will detect the language using the **country of this video**

In [ ]:
df["country"].unique()

In [ ]:
country_languages = {
    'DE': 'German',
    'US': 'English', 'PL': 'Polish',
    'SA': 'Arabic', 'NP': 'Nepali',
    'CA': 'English', 'ES': 'Spanish',
    'TR': 'Turkish', 'IN': 'Hindi',
    'EG': 'Arabic', 'GB': 'English',
    'MX': 'Spanish', 'BR': 'Portuguese',
    'PK': 'Urdu', 'FR': 'French',
    'VN': 'Vietnamese', 'ID': 'Indonesian',
    'AU': 'English', 'HU': 'Hungarian',
    'NL': 'Dutch', 'BG': 'Bulgarian',
    'JP': 'Japanese', 'SG': 'English', 
    'TH': 'Thai', 'PH': 'Tagalog',
    'MT': 'Maltese', 'PE': 'Spanish',
    'SE': 'Swedish', 'IT': 'Italian',
    'KR': 'Korean', 'TW': 'Chinese',
    'FI': 'Finnish', 'DZ': 'Arabic',
    'BD': 'Bengali', 'AR': 'Spanish'}

df["language"] = df["country"].replace(country_languages)

This way may not be so accurate in detecting languages becuase there are some **indian videos** specifically with english and<br>
because some commetns come in deffrint languages but at least this way is more accurate than `roberta-language-detection`<br>
model because this model sometimes come with wierd 

#### *Sentiment analysis*

In [ ]:
sentiment_classifier = transformers.pipeline(model= "finiteautomata/bertweet-base-sentiment-analysis")
print(sentiment_classifier("Hello here in my analysis, Have a nice day !!"))

In [ ]:
def my_sentiment_classifier(text: str) -> np.int8:
    """This code takes a text and return you if it's positive
        or negative as 1 for positive, 0 for natural -1 for negative
        and -10 for unclassified.
    
    @params: a string text
    @return: 1, 0 or -1 in np.int8 dtype"""
    
    sentiment_type_encoder: dict = {"POS": 1,
                                    "NEG": -1,
                                    "NEU": 0,
                                    "unclassified": -10}
    
    try:
        sentiment_type: str = sentiment_classifier(text)[0]["label"]
        
    except:
        sentiment_type: str = "unclassified"
        
    return  np.int8(sentiment_type_encoder[sentiment_type])

In [ ]:
df["sentiments"] = df["Comments"][:5].apply(
    lambda x: my_sentiment_classifier(x))

In [ ]:
df[["sentiments", "Comments"]]

#### *Video stats range*

We will classify each column of **[Comments, Likes, Views]** into 6 groups manually like:
<ul>
    <li>  $1\sim3,000$ Views
    <li>  $3,000\sim10,000$ Views
    <li>  $10,000\sim50,000$ Views
    <li>  $50,000\sim100,000$ Views
    <li>  $100,000\sim300,000$ Views
    <li>  $300,000\sim\infty$

We will see the max calues of each column so we can optimize it with the best dtype for it.

In [ ]:
print(f" The max value of view count column is: {df['viewCount'].astype(int).max()}\n",
      f"The max value of like count column ias: {df['likeCount'].astype(int).max()}\n",
      f"The max value of comment count column is: {df['commentCount'].astype(int).max()}\n",
      f"The max value of subscribers column is: {df['subscribers'].astype(int).max()}\n")

In [ ]:
df = df.astype({"commentCount": np.uint16, "viewCount": np.uint32,
                "likeCount": np.uint16, "subscribers": np.uint32})

df.describe()

In [ ]:
df["cat_view_count"] = pd.cut(df['viewCount'],
                         bins=[1, 3_000, 10_000, 50_000, 100_000, 300_000, 999_999_999_999],
                         labels=["from 1 to 3,000", "from 3,000 to 10,000",
                                 "from 10,000 to 50,000", "from 50,000 to 100,000",
                                 "from 100,000 to 300,000", "more than 300,000"])

df["cat_comment_count"] = pd.cut(df['commentCount'],
                         bins=[50, 100, 150, 200, 400, 600, 999_999_999_999],
                         labels=["from 50 to 100", "from 100 to 150",
                                 "from 150 to 200", "from 200 to 400",
                                 "from 400 to 600", "more than 600"])

df["cat_like_count"] = pd.cut(df['likeCount'],
                         bins=[0, 1_000, 5_000, 10_000, 50_000, 150_000, 999_999_999_999],
                         labels=["from 1 to 1,000", "from 1,000 to 5,000",
                                 "from 5,000 to 10,000", "from 10,000 to 50,000",
                                 "from 50,000 to 150,000", "more than 150,000"])

In [ ]:
df[["cat_view_count", "cat_comment_count", "cat_like_count",
    "viewCount", "commentCount", "likeCount"]].sample(5)

#### *Text columns length*

In [ ]:
for col in TEXT_COLUMNS:
    df[f"{col}_length"] = df[col].str.len()

## <center><strong>Text visualiztion with<span style = "color: red"> WordCloud</span></strong></center>

In [ ]:
coloring = np.array(Image.open("../imgs/youtube_gaming_logo.png"))
stopwords = set(STOPWORDS)

comments = ''.join(df["comments"][df["language"] == "English"].values.flatten())

In [ ]:
comments_wc = WordCloud(background_color= "white", max_words= 300, mask= coloring,
               stopwords= stopwords, max_font_size= 80, random_state= 42,
               font_path= '../Data analysis/assets/fonts/FranklinGothic.ttf',
               collocations=False)

image_colors = ImageColorGenerator(np.array(coloring))
comments_wc.generate(comments)
comments_wc = comments_wc.recolor(color_func= image_colors)

comments_svg = comments_wc.to_svg(embed_font=True)

# Save the SVG code to a file
with open("../plots/comments_word_cloud.svg", "w", encoding="utf-8") as f:
    f.write(comments_svg)
    
fig, ax = plt.subplots(1)

ax.imshow(comments_wc, interpolation="bilinear")
ax.set_axis_off()

plt.title("Comments word cloud")
fig.set_size_inches(10, 8)
plt.savefig("../plots/comments_word_cloud.png")
plt.show()

Now we will make a word cloud for **video titles data**

In [ ]:
popular_emojis = ["😊", "😍", "😘", "😜", "😝", "😂", "😭", "😡", "😠", "😩", "😫", "😞", "😟", "😢", "😥", "😰", "😱",
                       "😳", "😷", "👍", "👎", "👌", "👏", "🙌", "👋", "💪", "🙏", "❤️", "💔", "💕", "💖", "💘", "💙", "💚",
                       "💛", "💜", "💯", "🔥", "🌟", "✨", "⭐", "🌈", "🌺", "🍕", "🍔", "🍟", "🍦", "🍭", "🍩", "🍪", "🍺",
                       "🍻", "🍷", "🍸", "🎂", "🎁", "🎉", "🎊", "🎈", "🎵", "🎶", "🎼", "🎧", "🎤", "🎸", "🎹", "🎺", "🎷",
                       "🎻", "🎬", "🎥", "🎦", "📷", "📹", "📺", "📻", "💻", "📱", "💡", "🔑", "🔨", "🔥", "💰", "💳", "💼",
                       "📅", "📆", "📈", "📉", "📊", "📋", "📎", "📏", "📐", "🔒", "🔓", "🔍", "🔎", "🚀", "🚑", "🚒", "🚓",
                       "🚕", "🚗", "🚙", "🚚", "🚢", "🚤", "🚲", "🚶", "🚶‍♀️", "🏃", "🏃‍♀️", "⚽", "🏀", "🏈", "🎾", "🏐", "🏉",
                       "🎱", "🏓", "🏸", "🥊", "🥋", "🎮", "🕹️", "🎲", "🃏"]


for emoji in popular_emojis:
    stopwords.add(emoji)
    
titels = ''.join(df["title"][df["language"] == "English"].values.flatten())

In [ ]:
coloring = np.array(Image.open("../imgs/joystick.png"))

titels_wc = WordCloud(background_color= "white", max_words= 2000, mask= coloring,
               stopwords= stopwords, max_font_size= 80, random_state= 42,
               font_path= '../Data analysis/assets/fonts/FranklinGothic.ttf')

titels_wc.generate(titels)
image_colors = ImageColorGenerator(np.array(coloring))
titels_wc = titels_wc.recolor(color_func= image_colors)


fig, ax = plt.subplots(1)

ax.imshow(titels_wc, interpolation= "bilinear")
ax.set_axis_off()

titels_svg = titels_wc.to_svg(embed_font=True)

with open("../plots/videos_titels_word_cloud.svg", "w", encoding="utf-8") as f:
    f.write(titels_svg)

plt.title("Video titles word cloud")
fig.set_size_inches(10, 8)
plt.savefig("../plots/videos_titels_word_cloud.png")
plt.show()

## <center><strong><span style = "color: red">NLP</span> preprocessing

#### *Removing stop words*

In [ ]:
from nltk.corpus import stopwords
# nltk.download()

nltk.download('punkt', force=True)
nltk.download('wordnet', force= True)
nltk.download('stopwords', force=True)
nltk.download('stopwords-hi', force=True)
nltk.download('stopwords-ar', force=True)
nltk.download('averaged_perceptron_tagger', force=True)


en_stopwords = set(stopwords.words('english')) 
ar_stopwords = set(stopwords.words('arabic')) 
# hi_stopwords = set(stopwords.words('hindi')) 

all_stopwords = en_stopwords.union(ar_stopwords)

In [ ]:
def stopwords_dropper(words: list, stopwords: set) -> list:
    
    # Removing stop words from unalphabetical chars
    filtered_words = [re.sub(r"[\W_]", "", word) for word in words
                      if not word in stopwords]
    
    filtered_words = list(filter(lambda item: item is not "", filtered_words))
    return  filtered_words


for col in TEXT_COLUMNS:
    df[f"{col}_tokens"] = df[col].apply(lambda text: nltk.word_tokenize(text.lower()))
    df[f"{col}_tokens"] = df[f"{col}_tokens"].apply(lambda text: stopwords_dropper(text,
                                                                     all_stopwords))

df["comments_tokens"]

#### *Part-of-speech (POS) Tagging*

In [ ]:
for col in TEXT_COLUMNS:
    df[f"{col}_pos_tags"] = df[f"{col}_tokens"].apply(lambda words: nltk.pos_tag(words))

df["about_pos_tags"]

#### *Lemmatization and dropping duplicated words*

In [ ]:
def get_wordnet_pos(treebank_tag: str) -> str:
    
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = []
lemmatized_words_group = []

for col in TEXT_COLUMNS:
    for index, row in df.iterrows():
        for token, pos_tag in zip(row[f"{col}_tokens"], row[f"{col}_pos_tags"]):

            wordnet_pos = get_wordnet_pos(pos_tag[1])
            lemmatized_words_group.append(lemmatizer.lemmatize(token, pos= wordnet_pos))
            lemmatized_words_group = list(set(lemmatized_words_group)) # Dropping duplicates


        lemmatized_words.append(lemmatized_words_group)
        lemmatized_words_group = [] # clearing this list
    
    df[f"{col}_tokens"] = lemmatized_words
    lemmatized_words = []

In [ ]:
df["channelTitle_tokens"][:5]

#### *One hot encoding*

In [ ]:
df["language"] = df["language"].astype("category").cat.codes
df["definition"] = df["definition"].astype("category").cat.codes
# df["sentiments"] = df["sentiments"].astype("category")

df["cat_view_count"] = df["cat_view_count"].replace({"from 1 to 3,000": 1, "from 3,000 to 10,000": 2,
                                                     "from 10,000 to 50,000": 3, "from 50,000 to 100,000": 4,
                                                     "from 100,000 to 300,000": 5, "more than 300,000": 6})

df["cat_like_count"] = df["cat_like_count"].replace({"from 1 to 1,000": 1, "from 1,000 to 5,000": 2,
                                                     "from 5,000 to 10,000": 3, "from 10,000 to 50,000": 4,
                                                     "from 50,000 to 150,000": 5, "more than 150,000": 6})

df["cat_comment_count"] = df["cat_comment_count"].replace({"from 50 to 100": 1, "from 100 to 150": 2,
                                                           "from 150 to 200": 3, "from 200 to 400": 4,
                                                           "from 400 to 600": 5, "more than 600": 6})

In [ ]:
df[["language", "definition", "cat_view_count",
    "cat_like_count", "cat_comment_count"]].head()

## <center><strong><span style = "color: red">Text</span> classification

#### *Preparing training & testing data*

In [ ]:
X = df[["channelTitle_tokens", "comments_tokens"]].astype(str)


y = df[["cat_view_count", "cat_like_count"]]

# # Tokenize the text feature.
# text_tokens = df['comments_tokens'].apply(lambda x: ' '.join(x).split())

# Tokenize the other feature.
# other_tokens = df['other_feature'].apply(lambda x: x.split())

# idx = np.random.randint(4_000, size= 5)
# pd.DataFrame(X[idx,:])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 1/ 5, random_state= 99)

In [ ]:
print(X_train.shape)
print(y_train.shape)

#### *XGBoost classfier*

In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier()),
])

# Fit the model and make predictions
text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

# Evaluate model performance
print(classification_report(y_test, y_pred))